In [1]:
import pandas as pd
from tqdm import tqdm


train, valid, test = [], [], []
test_user_pool = set()
with open("test.txt", "r") as f:
    for line in tqdm(f.readlines()):
        uid, *test_seq = line.strip().split()
        if not test_seq:
            continue
        test.extend(
            [
                {"user_id": int(u)+1, "item_id": int(i)+1, "timestamp": 1}
                for u, i in zip([uid] * len(test_seq), test_seq)
            ]
        )
        test_user_pool.add(uid)
with open("train.txt", "r") as f:
    for line in tqdm(f.readlines()):
        uid, *seq = line.strip().split()
        if uid not in test_user_pool:
            print(uid, "SEQ:", seq)
            test_split = int(len(seq) * 0.8)
            seq, test_seq = seq[:test_split], seq[test_split:]
            test.extend(
                [
                    {"user_id": int(u)+1, "item_id": int(i)+1, "timestamp": 1}
                    for u, i in zip([uid] * len(test_seq), test_seq)
                ]
            )
        split_idx = int(len(seq) * 0.9)
        train_seq, valid_seq = seq[:split_idx], seq[split_idx:]
        train.extend(
            [
                {"user_id": int(u)+1, "item_id": int(i)+1, "timestamp": 1}
                for u, i in zip([uid] * len(train_seq), train_seq)
            ]
        )
        valid.extend(
            [
                {"user_id": int(u)+1, "item_id": int(i)+1, "timestamp": 1}
                for u, i in zip([uid] * len(valid_seq), valid_seq)
            ]
        )

train = pd.DataFrame(train)
valid = pd.DataFrame(valid)
test = pd.DataFrame(test)

print(train.groupby("user_id").count()["item_id"].describe())
print()
print(train.groupby("item_id").count()["user_id"].describe())

train.to_csv("train.csv", index=False)
valid.to_csv("valid.csv", index=False)
test.to_csv("test.csv", index=False)

100%|██████████| 14356/14356 [00:00<00:00, 58063.68it/s]


count    14356.000000
mean        18.100167
std         20.347254
min          7.000000
25%          8.000000
50%         11.000000
75%         19.000000
max        489.000000
Name: item_id, dtype: float64

count    15560.000000
mean        16.699614
std         16.461892
min          1.000000
25%          8.000000
50%         12.000000
75%         19.000000
max        308.000000
Name: user_id, dtype: float64


In [3]:
ks10 = pd.concat([train, valid, test], ignore_index=True)

In [4]:
ks10.to_csv("ks10.csv", index=False)


In [6]:
file_path = 'ks10.csv'
df = pd.read_csv(file_path)
df


,user_id,item_id,timestamp
0,1,8868,1
1,1,8869,1
2,1,761,1
3,1,3560,1
4,1,5585,1
...,...,...,...
367472,14356,15463,1
367473,14356,13830,1
367474,14356,15685,1
367475,14356,15806,1


In [11]:
import pandas as pd

df_u = df.drop_duplicates(subset='user_id', keep='first', ignore_index=True)

df_remap = pd.DataFrame({
    'org_id': df_u['user_id'],         # Original user_id
    'remap_id': df_u['user_id'] - 1    # Remap user_id starting from 0
})

output_file_path = 'user_list.txt'
df_remap.to_csv(output_file_path, sep='\t', index=False)

# Optionally, you can print the first few rows of the remapped DataFrame.
print(df_remap.head())


   org_id  remap_id
0       1         0
1       2         1
2       3         2
3       4         3
4       5         4


In [12]:
import pandas as pd

df_temp = df.drop_duplicates(subset='item_id', keep='first', ignore_index=True)

df_remap = pd.DataFrame({
    'org_id': df_temp['item_id'],         # Original item_id
    'remap_id': df_temp['item_id'] - 1    # Remap user_id starting from 0
})

output_file_path = 'item_list.txt'
df_remap.to_csv(output_file_path, sep=' ', index=False)

# Optionally, you can print the first few rows of the remapped DataFrame.
print(df_remap.head())


   org_id  remap_id
0    8868      8867
1    8869      8868
2     761       760
3    3560      3559
4    5585      5584


In [13]:
max(df['item_id'])

15885

In [14]:
max(df['user_id'])

14356

In [15]:
len(df)

367477

In [ ]:
COL_USER_ID = "user_id"
COL_ITEM_ID = "item_id"
COL_RATINGS = "rating"
COL_TIMESTAMP = "timestamp"

def get_sequence(ratings, num_users, num_items):
    ratings.sort_values(by=COL_TIMESTAMP, inplace=True)
    itemseqs = ratings.groupby(COL_USER_ID)[COL_ITEM_ID].apply(list).reset_index(name="items")
    user_itemseqs = {user_id: items for user_id, items in zip(itemseqs[COL_USER_ID], itemseqs["items"])}    
    userseqs = ratings.groupby(COL_ITEM_ID)[COL_USER_ID].apply(list).reset_index(name="users")
    item_userseqs = {item_id: users for item_id, users in zip(userseqs[COL_ITEM_ID], userseqs["users"])}    
    item_userseqs = {k: item_userseqs[k] if k in item_userseqs else [] for k in range(1, num_items + 1)}
    return (user_itemseqs, item_userseqs)

In [11]:
# user_id,item_id,rating,timestamp
import pandas as pd

train_path = 'test.csv'
valid_path = 'valid.csv'
test_path = 'test.csv'
train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)
train.head


<bound method NDFrame.head of         user_id  item_id  rating   timestamp
0          6034     1552       5   956703977
1          6034      501       4   956704056
2          6034      981       5   956704257
3          6034      209       5   956704613
4          6034      676       4   956704788
...         ...      ...     ...         ...
112423     5944     2219       4  1046368263
112424     5944     2901       4  1046368496
112425     5944     3093       4  1046369026
112426      403      818       4  1046388675
112427     3158     2082       4  1046393499

[112428 rows x 4 columns]>

<bound method NDFrame.head of         user_id  item_id  rating   timestamp  user_id_new
0          6034     1552       5   956703977         6033
1          6034      501       4   956704056         6033
2          6034      981       5   956704257         6033
3          6034      209       5   956704613         6033
4          6034      676       4   956704788         6033
...         ...      ...     ...         ...          ...
112423     5944     2219       4  1046368263         5943
112424     5944     2901       4  1046368496         5943
112425     5944     3093       4  1046369026         5943
112426      403      818       4  1046388675          402
112427     3158     2082       4  1046393499         3157

[112428 rows x 5 columns]>

In [14]:
import csv

train['user_id_new'] = train['user_id'] - 1
train.head
user_item_history = train.groupby('user_id_new')['item_id'].agg(lambda x: ' '.join(map(str, x.astype(int)))).reset_index()
# 
# user_item_history['item_id'] = user_item_history['item_id'].str.replace('"', '')
# user_item_history
output_file_path = 'temp.txt'
user_item_history.to_csv(output_file_path, sep=' ', index=False, header=False, quoting=0, escapechar=None)

# Read the data from the original file and modify the format
with open("temp.txt", "r") as infile, open("train.txt", "w") as outfile:
    for line in infile:
        label, values = line.strip().split(" ", 1)
        values_list = " ".join(values.replace('"', '').split())
        outfile.write(f"{label} {values_list}\n")

# print(user_item_history.head())


In [16]:
import csv

valid['user_id_new'] = valid['user_id'] - 1
valid.head
user_item_history = valid.groupby('user_id_new')['item_id'].agg(lambda x: ' '.join(map(str, x.astype(int)))).reset_index()
# 
# user_item_history['item_id'] = user_item_history['item_id'].str.replace('"', '')
# user_item_history
output_file_path = 'temp.txt'
user_item_history.to_csv(output_file_path, sep=' ', index=False, header=False, quoting=0, escapechar=None)

# Read the data from the original file and modify the format
with open("temp.txt", "r") as infile, open("valid.txt", "w") as outfile:
    for line in infile:
        label, values = line.strip().split(" ", 1)
        values_list = " ".join(values.replace('"', '').split())
        outfile.write(f"{label} {values_list}\n")

# print(user_item_history.head())


In [25]:
# user_id,item_id,rating,timestamp
import pandas as pd

train_path = 'train.csv'
valid_path = 'valid.csv'
test_path = 'test.csv'
train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)
train.head


<bound method NDFrame.head of         user_id  item_id  rating   timestamp
0          6034      670       4   956703932
1          6034     1863       4   956703954
2          6034      515       5   956703954
3          6034     1497       4   956703977
4          6034     2767       4   956704056
...         ...      ...     ...         ...
407179     5307     2588       4  1046444711
407180     4953     1928       4  1046454260
407181     4953     2753       4  1046454320
407182     4953     2585       4  1046454548
407183     4953     1461       4  1046454590

[407184 rows x 4 columns]>

In [26]:
import csv

train['user_id_new'] = train['user_id'] - 1
train.head
user_item_history = train.groupby('user_id_new')['item_id'].agg(lambda x: ' '.join(map(str, x.astype(int)))).reset_index()
# 
# user_item_history['item_id'] = user_item_history['item_id'].str.replace('"', '')
# user_item_history
output_file_path = 'temp.txt'
user_item_history.to_csv(output_file_path, sep=' ', index=False, header=False, quoting=0, escapechar=None)

# Read the data from the original file and modify the format
with open("temp.txt", "r") as infile, open("train.txt", "w") as outfile:
    for line in infile:
        label, values = line.strip().split(" ", 1)
        values_list = " ".join(values.replace('"', '').split())
        outfile.write(f"{label} {values_list}\n")

# print(user_item_history.head())


In [32]:
import pandas as pd

# Assuming you have 'train' DataFrame defined somewhere before this code.

# Group the 'item_id' values by 'user_id', convert them to integers, and concatenate them into a space-separated string.
# user_item_history = train.groupby('user_id')['item_id'].agg(lambda x: ' '.join(map(str, x.astype(int)))).reset_index()
user_item_history = train.groupby('user_id_new')['item_id'].apply(list).reset_index()
# user_item_history
output_file_path = 'train.txt'
# user_item_history.to_csv(output_file_path, sep=' ', index=False, header=False, quoting=0, escapechar=' ')

# Optionally, you can print the first few rows of the user_item_history DataFrame.
# print(user_item_history.head())


,user_id_new,item_id
0,0,"[804, 2686, 925, 527, 817, 125, 2181, 2309, 1201]"
1,1,"[2736, 1967, 2502, 2466, 1070, 1489, 716, 1498..."
2,2,"[1504, 941, 1253, 553, 826, 1033, 1836]"
3,3,"[865, 929, 1490]"
4,4,"[715, 2259, 2303, 2029, 709, 2146, 2022, 150, ..."
...,...,...
6029,6029,"[2043, 2029, 2136, 1804, 1495, 2178, 2181, 212..."
6030,6030,"[2259, 461, 273, 704, 527, 715, 2736, 934, 100..."
6031,6031,"[198, 954]"
6032,6032,"[942, 2522, 2675, 707, 920, 712, 2739, 941, 80..."


In [18]:
# user_id,item_id,rating,timestamp
import pandas as pd

train_path = 'test.csv'
valid_path = 'valid.csv'
test_path = 'test.csv'
train = pd.read_csv(train_path)
valid = pd.read_csv(valid_path)
test = pd.read_csv(test_path)
train.head


<bound method NDFrame.head of         user_id  item_id  rating   timestamp
0          6034     1552       5   956703977
1          6034      501       4   956704056
2          6034      981       5   956704257
3          6034      209       5   956704613
4          6034      676       4   956704788
...         ...      ...     ...         ...
112423     5944     2219       4  1046368263
112424     5944     2901       4  1046368496
112425     5944     3093       4  1046369026
112426      403      818       4  1046388675
112427     3158     2082       4  1046393499

[112428 rows x 4 columns]>

In [21]:
ratings = train
num_users = 10

COL_USER_ID = "user_id"
COL_ITEM_ID = "item_id"
COL_RATINGS = "rating"
COL_TIMESTAMP = "timestamp"

val_set_percentage = 0.2
test_set_percentage = 0.1

train_lst = []
valid_lst = []
test_lst = []
for user_id in (range(1, num_users + 1)):
        user_items = ratings[ratings[COL_USER_ID] == user_id]
        num_items = len(user_items)
        
        val_size = int(num_items * val_set_percentage)
        val_size = 1 if val_size == 0 else val_size
        test_size = int(num_items * test_set_percentage)
        test_size = 1 if test_size == 0 else test_size

        val_items = user_items.sample(n=val_size)
        remaining_items = user_items.drop(val_items.index)
        test_items = remaining_items.sample(n=test_size)
        train_items = remaining_items.drop(test_items.index)
        
        if not len(train_items):
            print(num_items, 'total')
            print(test_size, 'test')
            print(val_size, 'val')
            print(len(train_items), 'train')
            print(len(remaining_items), 'remain')
            raise ValueError   
        
        train_lst.append(train_items)
        valid_lst.append(val_items)
        test_lst.append(test_items)
        
        print('user_id', user_id)
        print(train_lst)
        print(valid_lst)
        print(test_lst)
        break

user_id 1
[        user_id  item_id  rating  timestamp
102192        1      804       5  978300055
102195        1     2686       4  978300275
102197        1      527       4  978301398
102198        1      817       5  978301753
102199        1      125       5  978301777
102200        1     2181       4  978302091
102201        1     2309       4  978302124]
[        user_id  item_id  rating  timestamp
102787        1     1201       4  978824139]
[        user_id  item_id  rating  timestamp
102196        1      925       5  978300760]
